In [26]:
#Setup inference and evaluation parameters
import re
import sys
import os
import random
GPU_ID="0,1"
os.environ["CUDA_VISIBLE_DEVICES"]=GPU_ID
LoadBaseOutput=False
Customer=1
CustomPrompt=1
BaseModelName='goliath-120b-AWQ'
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"]="python"
import Libs.DataSynthesisLib as DataSynthesisLib
Config=DataSynthesisLib.GetConfig('Config/Config.py')

ModelFT='./tmp/merged_AllComments_Clean_Trainr_128_a_64_d_0.05'
#ModelBase='unsloth/Meta-Llama-3.1-8B-Instruct'
ModelBase='TheBloke/'+BaseModelName


AssistantPromptElementsRemoval=2
EvalLLM='microsoft/phi-4'


In [3]:
#Load appropriate evaluation dataset
import json
import random


if Customer==1:
  with open('Data/CustomerComments_Evaluation_Clean.json', 'r') as file:
    data = json.load(file)
elif Customer==0:
  with open('Data/ClouderaComments_Evaluation_Clean.json', 'r') as file:
    data = json.load(file)





In [4]:
#Setup questions for LLM-as-a-judge, collect prompts for inference, and customer or Cloudera comments to be used later for evaluation
import vllm
from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained(ModelFT)

System='You are a helpful assistant.'


FixedQuestions={}
if Customer==1:
  FixedQuestions[1]="Does this comment discuss any technical information? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[2]="Does this comment relate to a customer complaint? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[3]="Customer complaint temperature or a frustration level (if there is a complain give a Score 1-4, lowest=1, highest=4. If there is no complain give a score of 0)"
  FixedQuestions[4]="Score the severity of the issue based on comment content (SCORE 1-4, lowest=1, highest=4)"
  FixedQuestions[5]="Score the urgency of the issue based on the comment content (SCORE 1-4, lowest=1, highest=4)"
  FixedQuestions[6]="Is this a request from a customer for an update? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[7]="Is there a strictly explicit and NOT an implied request from a customer for a call, meeting or a screenshare (zoom/webex/teams etc.)? Do not answer yes unless wording explicitly asks for a call. (BOOL)"
  FixedQuestions[8]="Did the customer request an escalation? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[9]="Did the customer request a priority change?  To what level? (If there is a priority change give score 1 to indicate highest priority (indicated by S1) and 4  to indicate the lowest priority (Indicated by 4). If there is no priority change give a score of 0)."
  FixedQuestions[10]="Did the customer request a transfer to another COE? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[11]="Did the customer request to speak to a manager or supervisor? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[12]="Did the customer request a SME or expert? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[13]="Does this comment discuss a bug in Cloudera software? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[14]="Does the comment include a non-Cloudera Apache JIRA link? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[15]="Does the comment have a link to Cloudera Documentation or Community article? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[16]="Does the comment have any other type of hyperlink? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[17]="Summarize the case comment condensing it as much as possible but without losing important technical details. Omit including any meeting invite information.  [TEXT]"
elif Customer==0:
  FixedQuestions[1]="Does this comment discuss any technical information? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[2]="Score the severity of the issue based on comment content (SCORE 1-4, lowest=1, highest=4)"
  FixedQuestions[3]="Score the urgency of the issue based on the comment content (SCORE 1-4, lowest=1, highest=4)"
  FixedQuestions[4]="Does the comment have a proposed solution? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[5]="Does the comment have a proposed workaround?  (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[6]="Does the comment have a request for an action from the customer?  (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[7]="Does this comment discuss a bug in Cloudera software? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[8]="Does the comment include a non-Cloudera Apache JIRA link? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[9]="Does the comment have a link to Cloudera Documentation or Community article? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[10]="Does the comment have any other type of hyperlink? (BOOL: 0 for no, 1 for yes)"
  FixedQuestions[11]="Summarize the case comment condensing it as much as possible but without losing important technical details. Omit including any meeting invite information.  [TEXT] "


Questions=[]
Comment=[]
BaselineSolutions=[]
for i in data:
  if Customer==1:
      
    if CustomPrompt==1:

      Questions.append(i['Prompt'])
      Comment.append(i['Comment'])

  elif Customer==0:

    CommentText="""You are given a Cloudera support team comment, 11 questions referring to the comment, and the possible values for each question in parenthesis following the question. Here is the Cloudera comment:

"""
    PreppendClouderaQuestions="""

Here are the 11 questions:
1. Does this comment discuss any technical information? (answer 0 for no, 1 for yes)
2. Score the severity of the issue based on comment content (SCORE 1-4, give 1 for lowest, 4 for highest and 2,3 for in between)
3. Score the urgency of the issue based on the comment content (SCORE 1-4, give 1 for lowest, 4 for highest and 2,3 for in between)
4. Does the comment have a proposed solution? (answer 0 for no, 1 for yes)
5. Does the comment have a proposed workaround?  (answer 0 for no, 1 for yes)
6. Does the comment have a request for an action from the customer?  (answer 0 for no, 1 for yes)
7. Does this comment discuss a bug in Cloudera software? (answer 0 for no, 1 for yes)
8. Does the comment include a non-Cloudera Apache JIRA link (e.g. a Apache JIRA link with issues.apach.org domain name)? (answer 0 for no, 1 for yes)
9. Does the comment have a link to Cloudera Documentation or Community article? (answer 0 for no, 1 for yes)
10. Does the comment have any other type of hyperlink? (answer 0 for no, 1 for yes)
11. Summarize the case comment condensing it as much as possible but without losing important technical details. Omit including any meeting invite information.  [TEXT]    
Generate each answer in a new line using the the format "Number. answer" (e.g. 1. 0)
"""
    if CustomPrompt==1:
      Questions.append(i['Prompt'])
      Comment.append(i['Comment'])


print(len(Comment))
print(len(Questions))
lens=[len(i) for i in Questions ]

500
500


In [5]:
#Typical example on how the LLM is prompted
print(Questions[0])

You are given a customer comment, 17 questions referring to the comment, and the possible values for each question in parenthesis following the question. Here is the customer comment:

Hi Cloudera Support,\n\nI need urgent assistance with account authorization for our CDP production environment. I'm getting 'Authorization Error: Token Invalid' messages when trying to access critical resources, and none of our automation tools are working. This seems to have started after our scheduled maintenance window last night. We have multiple production jobs failing because of this.\n\nPlease help resolve this as soon as possible.\n\nBest regards,\nMark

Here are the 17 questions:
1. Does this comment discuss any technical information? (answer 0 for no, 1 for yes)
2. Does this comment relate to a customer complaint? (answer 0 for no, 1 for yes)
3. Customer complaint temperature or a frustration level (if there is a complain give 1 for lowest, 4 for highest and 2,3 for in between. If there is no c

In [27]:
if LoadBaseOutput==False:
  #Base model parameters setup and inference 
  from torch.multiprocessing import Pool, Process, set_start_method
  try:
     set_start_method('spawn')
  except RuntimeError:
    pass
  os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"
  ConfigS=Config["ConfigSolution"].copy()
  ConfigS["gpu-id"]=GPU_ID
  ConfigS["tensor_parallel_size"]="2"
  #ConfigS["tensor_parallel_size"]="1"

  ConfigS["system"]=System
  ConfigS["use-grammar"]="False"
  ConfigS["AssistantPrompt"]=""

  ConfigS["max_tokens"]="8000"
  ConfigS["temperature"]="0.1"
  ConfigS["max_num_seqs"]="2"

  ConfigS["max_num_seqs"]="16"
  ConfigS["max_model_len"]="32000"
  ConfigS["max_num_batched_tokens"]="32000"
 
  ConfigS["max_model_len"]="4000"
  ConfigS["max_num_batched_tokens"]="4000"


  ConfigS["AssistantPromptElementsRemoval"]="1"
  ConfigS["llm-path"]=ModelBase
  ConfigS["tokenizer-path"]=ModelBase

  ConfigS["tokenizer-file-gguf"]=""
  ConfigS["temperature"]="0"
  if "mistral" in ModelBase:
    ConfigS["GenerateChatTemplate"]="GenerateChatWithoutSystemWithAssistant"
  else:
    ConfigS["GenerateChatTemplate"]="GenerateChatWithSystem"

  ConfigS["UseOutlines"]="False"
  ConfigS["AssistantPromptElementsRemoval"]="1"
  ConfigS["tokenizer-file-gguf"]=""
  if 'gptq' in ModelFT:
    ConfigS["quantization"]="gptq"
  ConfigS["gpu_memory_utilization"]="0.99"
  ConfigS['chat_template']="{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% elif true == true and not '<<SYS>>' in messages[0]['content'] %}{% set loop_messages = messages %}{% set system_message = 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.\\n\\nIf a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don\\'t know the answer to a question, please don\\'t share false information.' %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if loop.index0 == 0 and system_message != false %}{% set content = '<<SYS>>\\n' + system_message + '\\n<</SYS>>\\n\\n' + message['content'] %}{% else %}{% set content = message['content'] %}{% endif %}{% if message['role'] == 'user' %}{{ bos_token + '[INST] ' + content.strip() + ' [/INST]' }}{% elif message['role'] == 'system' %}{{ '<<SYS>>\\n' + content.strip() + '\\n<</SYS>>\\n\\n' }}{% elif message['role'] == 'assistant' %}{{ ' '  + content.strip() + ' ' + eos_token }}{% endif %}{% endfor %}"


  (SolutionsBase, SolutionsBaseLogging)=DataSynthesisLib.GetSolutions(ConfigS, Questions, ConfigS["system"])


/home/cdsw/.local/lib/python3.10/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


TheBloke/goliath-120b-AWQ
INFO 03-24 17:49:09 awq_marlin.py:97] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
INFO 03-24 17:49:09 config.py:887] Defaulting to use mp for distributed inference
WARNING 03-24 17:49:09 config.py:380] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 03-24 17:49:09 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='TheBloke/goliath-120b-AWQ', speculative_config=None, tokenizer='TheBloke/goliath-120b-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=True, kv_cache_dtype=auto, qua

/home/cdsw/.local/lib/python3.10/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


(VllmWorkerProcess pid=3046) INFO 03-24 17:49:19 multiproc_worker_utils.py:216] Worker ready; awaiting tasks
INFO 03-24 17:49:19 utils.py:1008] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=3046) INFO 03-24 17:49:19 utils.py:1008] Found nccl from library libnccl.so.2
INFO 03-24 17:49:19 pynccl.py:63] vLLM is using nccl==2.20.5
(VllmWorkerProcess pid=3046) INFO 03-24 17:49:19 pynccl.py:63] vLLM is using nccl==2.20.5
(VllmWorkerProcess pid=3046) INFO 03-24 17:49:19 custom_all_reduce_utils.py:242] reading GPU P2P access cache from /home/cdsw/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
INFO 03-24 17:49:19 custom_all_reduce_utils.py:242] reading GPU P2P access cache from /home/cdsw/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
INFO 03-24 17:49:19 shm_broadcast.py:241] vLLM message queue communication handle: Handle(connect_ip='127.0.0.1', local_reader_ranks=[1], buffer=<vllm.distributed.device_communicators.shm_broadcast.ShmRingBuffer object at 0x7fa760f0fcd0>, local_subscri

Loading safetensors checkpoint shards:   0% Completed | 0/7 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  14% Completed | 1/7 [00:01<00:09,  1.53s/it]
Loading safetensors checkpoint shards:  29% Completed | 2/7 [00:03<00:07,  1.56s/it]
Loading safetensors checkpoint shards:  43% Completed | 3/7 [00:04<00:06,  1.52s/it]
Loading safetensors checkpoint shards:  57% Completed | 4/7 [00:06<00:04,  1.49s/it]
Loading safetensors checkpoint shards:  71% Completed | 5/7 [00:06<00:02,  1.10s/it]
Loading safetensors checkpoint shards:  86% Completed | 6/7 [00:07<00:01,  1.20s/it]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:09<00:00,  1.32s/it]
Loading safetensors checkpoint shards: 100% Completed | 7/7 [00:09<00:00,  1.34s/it]



INFO 03-24 17:49:33 model_runner.py:1071] Loading model weights took 28.9056 GB
(VllmWorkerProcess pid=3046) INFO 03-24 17:49:33 model_runner.py:1071] Loading model weights took 28.9056 GB
INFO 03-24 17:49:41 distributed_gpu_executor.py:57] # GPU blocks: 3370, # CPU blocks: 0
INFO 03-24 17:49:41 distributed_gpu_executor.py:61] Maximum concurrency for 4000 tokens per request: 13.48x


Processed prompts: 100%|██████████| 500/500 [13:46<00:00,  1.65s/it, est. speed input: 508.30 toks/s, output: 1.01 toks/s]


INFO 03-24 18:03:31 multiproc_worker_utils.py:134] Terminating local vLLM worker processes
(VllmWorkerProcess pid=3046) INFO 03-24 18:03:31 multiproc_worker_utils.py:242] Worker exiting


[rank0]:[W324 18:03:32.399122799 CudaIPCTypes.cpp:16] Producer process has been terminated before all shared CUDA tensors released. See Note [Sharing CUDA tensors]


In [23]:
if LoadBaseOutput==False:
  import  pickle
  if Customer==1:
    with open('Data/Eval_Customer_Questions_Solutions_'+BaseModelName+'.pickle', 'wb') as handle:
      pickle.dump({"Q":Questions, "S":SolutionsBase}, handle)
  elif Customer==0:
    with open('Data/Eval_Cloudera_Questions_Solutions_'+BaseModelName+'.pickle', 'wb') as handle:
      pickle.dump({"Q":Questions, "S":SolutionsBase}, handle)


In [14]:
if LoadBaseOutput==True:

  import  pickle
  if Customer==1:
    with open('Data/Eval_Customer_Questions_Solutions_'+BaseModelName+'.pickle', 'rb') as handle:
      Output=pickle.load( handle)
  elif Customer==0:
    with open('Data/Eval_Cloudera_Questions_Solutions_'+BaseModelName+'.pickle', 'rb') as handle:
      Output=pickle.load( handle)


  SolutionsBase=Output["S"]

In [24]:
Output["S"]

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '\n\n1. 1. 1\n2. 2. 0\n3. 3. 0\n4. 4. 2\n5. 5. 3\n6. 6. 0\n7. 7. 1\n8. 8. 0\n9. 9. 0\n10. 10. 0\n11. 11. 0\n12. 12. 0\n13. 13. 0\n14. 14. 0\n15. 15. 0\n16. 16. 0\n17. 17. Request for a meeting to discuss Ranger policies implementation and role-based access control.',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 

In [15]:
#Finetuned model parameters setup and inference 

from torch.multiprocessing import Pool, Process, set_start_method
try:
     set_start_method('spawn')
except RuntimeError:
    pass
os.environ["VLLM_WORKER_MULTIPROC_METHOD"] = "spawn"
ConfigS=Config["ConfigSolution"].copy()
ConfigS["gpu-id"]=GPU_ID
ConfigS["tensor_parallel_size"]="2"
#ConfigS["tensor_parallel_size"]="1"

ConfigS["system"]=System
ConfigS["use-grammar"]="False"
ConfigS["AssistantPrompt"]=""

ConfigS["max_tokens"]="3000"
ConfigS["temperature"]="0.1"
ConfigS["max_num_seqs"]="2"

ConfigS["max_num_seqs"]="16"
ConfigS["max_model_len"]="32000"
ConfigS["max_num_batched_tokens"]="32000"


ConfigS["AssistantPromptElementsRemoval"]="1"
ConfigS["llm-path"]=ModelFT
ConfigS["tokenizer-path"]=ModelFT

ConfigS["tokenizer-file-gguf"]=""
ConfigS["temperature"]="0"
if "mistral" in ModelFT:

  ConfigS["GenerateChatTemplate"]="GenerateChatWithoutSystemWithAssistant"
else:
  ConfigS["GenerateChatTemplate"]="GenerateChatWithSystem"

ConfigS["UseOutlines"]="False"
ConfigS["AssistantPromptElementsRemoval"]="1"
ConfigS["tokenizer-file-gguf"]=""
if 'gptq' in ModelFT:
  ConfigS["quantization"]="gptq"
ConfigS["gpu_memory_utilization"]="0.7"


(Solutions, SolutionsLogging)=DataSynthesisLib.GetSolutions(ConfigS, Questions, ConfigS["system"])


/home/cdsw/.local/lib/python3.10/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


./tmp/merged_AllComments_Clean_Trainr_128_a_64_d_0.05
INFO 03-24 17:34:38 config.py:887] Defaulting to use mp for distributed inference
WARNING 03-24 17:34:38 config.py:380] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 03-24 17:34:38 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='./tmp/merged_AllComments_Clean_Trainr_128_a_64_d_0.05', speculative_config=None, tokenizer='./tmp/merged_AllComments_Clean_Trainr_128_a_64_d_0.05', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_confi

/home/cdsw/.local/lib/python3.10/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION


(VllmWorkerProcess pid=2756) INFO 03-24 17:34:47 multiproc_worker_utils.py:216] Worker ready; awaiting tasks
INFO 03-24 17:34:48 utils.py:1008] Found nccl from library libnccl.so.2
(VllmWorkerProcess pid=2756) INFO 03-24 17:34:48 utils.py:1008] Found nccl from library libnccl.so.2
INFO 03-24 17:34:48 pynccl.py:63] vLLM is using nccl==2.20.5
(VllmWorkerProcess pid=2756) INFO 03-24 17:34:48 pynccl.py:63] vLLM is using nccl==2.20.5
INFO 03-24 17:34:48 custom_all_reduce_utils.py:242] reading GPU P2P access cache from /home/cdsw/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
(VllmWorkerProcess pid=2756) INFO 03-24 17:34:48 custom_all_reduce_utils.py:242] reading GPU P2P access cache from /home/cdsw/.cache/vllm/gpu_p2p_access_cache_for_0,1.json
INFO 03-24 17:34:48 shm_broadcast.py:241] vLLM message queue communication handle: Handle(connect_ip='127.0.0.1', local_reader_ranks=[1], buffer=<vllm.distributed.device_communicators.shm_broadcast.ShmRingBuffer object at 0x7f6865ae3100>, local_subscri

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  1.85it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:00<00:00,  2.92it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:07<00:03,  3.27s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:13<00:00,  4.42s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:13<00:00,  3.42s/it]



(VllmWorkerProcess pid=2756) INFO 03-24 17:35:02 model_runner.py:1071] Loading model weights took 7.5122 GB
INFO 03-24 17:35:02 model_runner.py:1071] Loading model weights took 7.5122 GB
INFO 03-24 17:35:10 distributed_gpu_executor.py:57] # GPU blocks: 21256, # CPU blocks: 0
INFO 03-24 17:35:10 distributed_gpu_executor.py:61] Maximum concurrency for 32000 tokens per request: 10.63x


Processed prompts: 100%|██████████| 500/500 [01:56<00:00,  4.29it/s, est. speed input: 3248.07 toks/s, output: 529.82 toks/s]


INFO 03-24 17:37:10 multiproc_worker_utils.py:134] Terminating local vLLM worker processes
(VllmWorkerProcess pid=2756) INFO 03-24 17:37:10 multiproc_worker_utils.py:242] Worker exiting


[rank0]:[W324 17:37:11.078179900 CudaIPCTypes.cpp:16] Producer process has been terminated before all shared CUDA tensors released. See Note [Sharing CUDA tensors]


In [22]:
#Typical example on how the LLM is prompted
print(Questions[43])

You are given a customer comment, 17 questions referring to the comment, and the possible values for each question in parenthesis following the question. Here is the customer comment:

Hello Cloudera Support, we've been experiencing some issues with our current Silver Support plan and would like to understand the benefits of upgrading to Gold Support. Our team needs faster response times for critical issues, and we're wondering if Gold Support would provide 24/7 coverage. Also, what's the typical timeline for implementing this change? Looking forward to your response.

Here are the 17 questions:
1. Does this comment discuss any technical information? (answer 0 for no, 1 for yes)
2. Does this comment relate to a customer complaint? (answer 0 for no, 1 for yes)
3. Customer complaint temperature or a frustration level (if there is a complain give 1 for lowest, 4 for highest and 2,3 for in between. If there is no complain give a score of 0).
4. Score the severity of the issue based on comm

NameError: name 'QuestionsBase' is not defined

In [21]:
#Example finetuned model output
print(Solutions[43])

<|start_header_id|>assistant<|end_header_id|>

1. 1
2. 0
3. 0
4. 2
5. 2
6. 1
7. 0
8. 0
9. 0
10. 0
11. 0
12. 0
13. 0
14. 0
15. 0
16. 0
17. Customer inquiring about benefits of upgrading from Silver to Gold Support, specifically regarding response times, 24/7 coverage, and implementation timeline.


In [20]:
#Example finetuned model output
print(SolutionsBase[42])

In [11]:
#Individual questions for evaluating each question using LLM-as-a-judge
FixedQuestions

{1: 'Does this comment discuss any technical information? (BOOL: 0 for no, 1 for yes)',
 2: 'Does this comment relate to a customer complaint? (BOOL: 0 for no, 1 for yes)',
 3: 'Customer complaint temperature or a frustration level (if there is a complain give a Score 1-4, lowest=1, highest=4. If there is no complain give a score of 0)',
 4: 'Score the severity of the issue based on comment content (SCORE 1-4, lowest=1, highest=4)',
 5: 'Score the urgency of the issue based on the comment content (SCORE 1-4, lowest=1, highest=4)',
 6: 'Is this a request from a customer for an update? (BOOL: 0 for no, 1 for yes)',
 7: 'Is there a strictly explicit and NOT an implied request from a customer for a call, meeting or a screenshare (zoom/webex/teams etc.)? Do not answer yes unless wording explicitly asks for a call. (BOOL)',
 8: 'Did the customer request an escalation? (BOOL: 0 for no, 1 for yes)',
 9: 'Did the customer request a priority change?  To what level? (If there is a priority change

In [12]:
#Parse LLM output and extract analytics and summary
def FormatOutput(Solutions,StartLine=3,ExpectedQuestions=17):
  CorrectFormat=0
  Output=[]
  Correct=[]

  for d in Solutions:
    result=d.split('\n')
    QuestionsAnswers={}
    count=0
    CorrectFormat=0
      
    for j in result[StartLine-1:]:
      FoundNumber=0
      count+=1
      context=""
      number=j.split('.')[0]
      if number.isdigit():
          number=int(number)
          if number==count:
              FoundNumber=1
      tmp=j.split(' ')
      if len(tmp)>=2:
         if count<ExpectedQuestions:
             if tmp[1].isdigit():
                 context=int(tmp[1])
                 if FoundNumber==1:
                    CorrectFormat+=1
                    QuestionsAnswers[number]=context
         else:
             context=' '.join(tmp[1:])
             if FoundNumber==1:
                CorrectFormat+=1
                QuestionsAnswers[number]=context
    Correct.append(CorrectFormat==ExpectedQuestions)
    Output.append(QuestionsAnswers)
  return (Output,Correct)

StartLine=3
if "Qwen2.5" in ModelFT:
    StartLine=2
if "mistral" in ModelFT:
    StartLine=1

if Customer==1:
  (FTOutput,FTCorrectFormat)=FormatOutput(Solutions,StartLine=StartLine)
elif Customer==0:
  (FTOutput,FTCorrectFormat)=FormatOutput(Solutions,StartLine=StartLine,ExpectedQuestions=11)



In [13]:
#Parsed formatted output for the finetuned LLM
print(FTOutput[12])
print(len(FTOutput))
print(len(FTCorrectFormat))
sum(FTCorrectFormat)

{1: 1, 2: 0, 3: 0, 4: 2, 5: 2, 6: 1, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 'Customer inquiring about discrepancy between advertised 35% CDP Data Hub promotion and actual 20% discount rate shown in their account portal, requesting verification of correct promotional rate for 400-node environment deployment.'}
500
500


498

In [14]:
#Parsed formatted output for the finetuned LLM
Solutions[2].split('\n')[2:]

['1. 1',
 '2. 0',
 '3. 0',
 '4. 2',
 '5. 2',
 '6. 0',
 '7. 0',
 '8. 0',
 '9. 0',
 '10. 0',
 '11. 0',
 '12. 0',
 '13. 0',
 '14. 0',
 '15. 0',
 '16. 0',
 '17. Customer inquiring about using NVMe drives in HP Proliant DL380 Gen10 servers for Cloudera cluster deployment focused on real-time analytics, requesting recommendations for number of NVMe drives per node for optimal performance.']

In [15]:
StartLine=3
if "Qwen2.5" in ModelBase:
    StartLine=2
if "mistral" in ModelBase:
    StartLine=1

if Customer==1:
  (BaselineOutput,BaselineCorrectFormat)=FormatOutput(SolutionsBase,StartLine=StartLine)
elif Customer==0:
  (BaselineOutput,BaselineCorrectFormat)=FormatOutput(SolutionsBase,StartLine=StartLine,ExpectedQuestions=11)



In [16]:
#Parsed formatted output for the base LLM

print(BaselineOutput[2])
print(len(BaselineOutput))
print(len(BaselineCorrectFormat))
sum(BaselineCorrectFormat)

{}
500
500


0

In [17]:
#Prompts and 3 examples for LLM-as-a-judge to score whether model A or B wins or if there is a tie.


comment="Hi Support team, we're experiencing critical performance issues with our Hive queries in our production environment. All queries are taking 3-4 times longer than usual to execute, severely impacting our business operations. Can you please escalate this to a high priority case and have someone look into this ASAP? We need immediate assistance as this is affecting our SLAs."
question="""Does this comment relate to a customer complaint? (BOOL)"""
AnswerA="""0"""
AnswerB="""1"""

Example1Prompt="You are given a comment and a question about the comment. You will be given the response of two systems and you need to judge which system  gave the best response. Make sure you provide the answer in json and your choice is given in a field with the value choice. Here is the comment: \n" + comment +".\nHere is the question:\n" + question+":\n" +"\nHere is the answer of System A:\n"+ AnswerA +"\nHere is the answer of System B:\n"+AnswerB

Example1Completion="""```json
{
    "choice": "B"
}
```
"""

comment="We're experiencing issues with our Cloudera Data Warehouse environment where our Impala queries are failing with OOM errors. The cluster metrics show high memory utilization across all nodes. This is blocking our critical reporting pipeline. Could you please help investigate this issue? We need this resolved within the next 24 hours."
question="""Customer complaint temperature or a frustration level (score 1-4, 0 if not a complaint)"""
AnswerA="""3"""
AnswerB="""3"""

Example2Prompt="You are given a comment and a question about the comment. You will be given the response of two systems and you need to judge which system  gave the best response. Make sure you provide the answer in json and your choice is given in a field with the value choice. Here is the comment: \n" + comment +".\nHere is the question:\n" + question+":\n" +"\nHere is the answer of System A:\n"+ AnswerA +"\nHere is the answer of System B:\n"+AnswerB

Example2Completion="""```json
{
    "choice": "tie"
}
```
"""
comment="Our Cloudera Manager is showing multiple critical alerts for HDFS data nodes. The replication factor has dropped below the minimum threshold, and we're seeing increased latency in our data processing jobs. Can you please help us resolve this urgently? This is impacting our production workloads."
question="""Summarize the case comment condensing it as much as possible but without losing important technical details. Omit including any meeting invite information.  [TEXT]"""
AnswerA="""Critical HDFS issues with data nodes showing alerts, reduced replication factor causing increased latency in production data processing jobs."""
AnswerB="""This is about critical alets."""

Example3Prompt="You are given a comment and a question about the comment. You will be given the response of two systems and you need to judge which system  gave the best response. Make sure you provide the answer in json and your choice is given in a field with the value choice. Here is the comment: \n" + comment +".\nHere is the question:\n" + question+":\n" +"\nHere is the answer of System A:\n"+ AnswerA +"\nHere is the answer of System B:\n"+AnswerB
Example3Completion="""```json
{
    "choice": "A"
}
```
"""



In [18]:

chat2 = [
  {"role": "system", "content": System},

  {"role": "user", "content": Example1Prompt},
  {"role": "assistant", "content": Example1Completion},
  {"role": "user", "content": Example2Prompt},
  {"role": "assistant", "content": Example2Completion},
  {"role": "user", "content": Example3Prompt},
  {"role": "assistant", "content": Example3Completion}
]

Completion="""```json
{
    "choice": """

chat=chat2.copy()


In [19]:
#Setup LLM-as-a-judge final prompt
from transformers import AutoTokenizer
EvalLLM='microsoft/phi-4'
RemoveAssistant=2
if EvalLLM=='microsoft/phi-4':
    RemoveAssistant=1
elif EvalLLM=='Qwen/Qwen2.5-32B-Instruct-GPTQ-Int8':
    RemoveAssistant=2
#tokenizer = AutoTokenizer.from_pretrained(EvalLLM)
tokenizer = AutoTokenizer.from_pretrained(EvalLLM)

if Customer==1:   
    ExpectedQuestions=17

elif Customer==0:
    ExpectedQuestions=11
    
FTCorrectFormat={}
BaselineCorrectFormat={}
AllPrompts=[]

for i in range(1,ExpectedQuestions+1):
    FTCorrectFormat[i]={}
    BaselineCorrectFormat[i]={}
    Prompts=[]
    print(i)
    for j in range(0,len(FTOutput)):
        chat=chat2.copy()
        
        if i in FTOutput[j]:
            FTCandidate=str(FTOutput[j][i])
            FTCorrectFormat[i][j]=True
        else:
            FTCandidate=""
            FTCorrectFormat[i][j]=False
            
        if i in BaselineOutput[j]:
            BaselineCandidate=str(BaselineOutput[j][i])
            BaselineCorrectFormat[i][j]=True
        else:
            BaselineCandidate=""
            BaselineCorrectFormat[i][j]=False
        user=""
        if j%2==0:
          user="You are given a comment and a question about the comment. You will be given the response of two systems, system A and system B and you need to judge which system  gave the best response or if it was a tie. Make sure you provide the answer in json and your choice is given in a field with the value choice. Here is the comment: \n" + Comment[j]+".\nHere is the question:\n"+FixedQuestions[i]+"\nHere is the answer of System A:\n" + FTCandidate +"\nHere is the answer of System B:\n"+BaselineCandidate + "\n"
        else:
          user="You are given a comment and a question about the comment. You will be given the response of two systems, system A and system B and you need to judge which system  gave the best response or if it was a tie. Make sure you provide the answer in json and your choice is given in a field with the value choice. Here is the comment: \n" + Comment[j]+".\nHere is the question:\n"+FixedQuestions[i]+"\nHere is the answer of System A:\n" + BaselineCandidate +"\nHere is the answer of System B:\n"+FTCandidate + "\n"

        chat.append({"role": "user", "content": user})
        chat.append({"role": "assistant", "content": Completion})
        
        Prompt=tokenizer.apply_chat_template(chat, tokenize=False)
        Prompt=tokenizer.decode(tokenizer.encode(Prompt)[:-RemoveAssistant])
        Prompts.append(Prompt)
    AllPrompts.append(Prompts)
    


print(tokenizer.encode(Prompts[1]))

        
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
[100264, 9125, 100266, 2675, 527, 264, 11190, 18328, 13, 100265, 100264, 882, 100266, 2675, 527, 2728, 264, 4068, 323, 264, 3488, 922, 279, 4068, 13, 1472, 690, 387, 2728, 279, 2077, 315, 1403, 6067, 323, 499, 1205, 311, 11913, 902, 1887, 220, 6688, 279, 1888, 2077, 13, 7557, 2771, 499, 3493, 279, 4320, 304, 3024, 323, 701, 5873, 374, 2728, 304, 264, 2115, 449, 279, 907, 5873, 13, 5810, 374, 279, 4068, 25, 720, 13347, 9365, 2128, 11, 584, 2351, 25051, 9200, 5178, 4819, 449, 1057, 69278, 20126, 304, 1057, 5788, 4676, 13, 2052, 20126, 527, 4737, 220, 18, 12, 19, 3115, 5129, 1109, 13783, 311, 9203, 11, 35906, 74055, 1057, 2626, 7677, 13, 3053, 499, 4587, 89690, 420, 311, 264, 1579, 10844, 1162, 323, 617, 4423, 1427, 1139, 420, 67590, 30, 1226, 1205, 14247, 13291, 439, 420, 374, 28987, 1057, 17216, 2170, 35047, 8586, 374, 279, 3488, 512, 22186, 420, 4068, 29243, 311, 264, 6130, 12458, 30, 320, 10611, 7887, 8586, 374, 279, 4320, 315, 744, 362, 512, 

In [20]:
#LLM-as-a-judge prompt example
print(AllPrompts[0][3])

<|im_start|>system<|im_sep|>You are a helpful assistant.<|im_end|><|im_start|>user<|im_sep|>You are given a comment and a question about the comment. You will be given the response of two systems and you need to judge which system  gave the best response. Make sure you provide the answer in json and your choice is given in a field with the value choice. Here is the comment: 
Hi Support team, we're experiencing critical performance issues with our Hive queries in our production environment. All queries are taking 3-4 times longer than usual to execute, severely impacting our business operations. Can you please escalate this to a high priority case and have someone look into this ASAP? We need immediate assistance as this is affecting our SLAs..
Here is the question:
Does this comment relate to a customer complaint? (BOOL):

Here is the answer of System A:
0
Here is the answer of System B:
1<|im_end|><|im_start|>assistant<|im_sep|>```json
{
    "choice": "B"
}
```<|im_end|><|im_start|>us

In [21]:
#Load LLM-as-a-judge
max_num_seqs=1
max_model_len=10000
max_num_batched_tokens=10000 
gpu_memory_utilization=0.7
enforce_eager=True
tensor_parallel_size=1
seed=None
max_tokens=10
temperature=0.1

import vllm
llm = vllm.LLM(model=EvalLLM, max_num_seqs=max_num_seqs,max_model_len=max_model_len,max_num_batched_tokens=max_num_batched_tokens, gpu_memory_utilization=gpu_memory_utilization,enforce_eager=enforce_eager, tensor_parallel_size=tensor_parallel_size)
seed=None
max_tokens=max_tokens
temperature=temperature
sampling_params = vllm.SamplingParams(seed=None,max_tokens=max_tokens,temperature=temperature)


WARNING 03-22 08:42:17 config.py:380] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 03-22 08:42:17 llm_engine.py:237] Initializing an LLM engine (vdev) with config: model='microsoft/phi-4', speculative_config=None, tokenizer='microsoft/phi-4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=10000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False)

Loading safetensors checkpoint shards:   0% Completed | 0/6 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  17% Completed | 1/6 [00:00<00:03,  1.40it/s]
Loading safetensors checkpoint shards:  33% Completed | 2/6 [00:01<00:02,  1.34it/s]
Loading safetensors checkpoint shards:  50% Completed | 3/6 [00:02<00:02,  1.36it/s]
Loading safetensors checkpoint shards:  67% Completed | 4/6 [00:02<00:01,  1.38it/s]
Loading safetensors checkpoint shards:  83% Completed | 5/6 [00:03<00:00,  1.39it/s]
Loading safetensors checkpoint shards: 100% Completed | 6/6 [00:04<00:00,  1.42it/s]
Loading safetensors checkpoint shards: 100% Completed | 6/6 [00:04<00:00,  1.40it/s]



INFO 03-22 08:42:25 model_runner.py:1071] Loading model weights took 27.3875 GB
INFO 03-22 08:42:27 gpu_executor.py:122] # GPU blocks: 702, # CPU blocks: 1310
INFO 03-22 08:42:27 gpu_executor.py:126] Maximum concurrency for 10000 tokens per request: 1.12x


In [22]:
#Run LLM-as-a-judge for eacb question and at the same time count correct answers/ties.
Outs=[]
PosPercNoTies=[]
NegPercNoTies=[]
PosPercTies=[]
NegPercTies=[]
countPosArr=[]
countNegArr=[]
countTieArr=[]
TiesPerc=[]

for j in range(0,ExpectedQuestions):
  max_num_seqs=16

  sampling_params = vllm.SamplingParams(seed=1,max_tokens=max_tokens,temperature=temperature)

  Out=llm.generate(AllPrompts[j], sampling_params)
  Outs.append(Out)
  countPos=0
  countNeg=0
  countTie=0

  for i in range(0,len(Out)):
    if i%2==0:
      #print(i)
      #print(Out[i].prompt)
      #print(Out[i].outputs[0].text[2])
      if len(Out[i].outputs[0].text)<2:
        continue

      if Out[i].outputs[0].text[2]=='A':
        countPos+=1
      elif Out[i].outputs[0].text[2]=='B':
        countNeg+=1
      elif Out[i].outputs[0].text[2]=='t':
        countTie+=1

    if i%2==1:
      if len(Out[i].outputs[0].text)<2:
        continue

      if Out[i].outputs[0].text[2]=='B':
        countPos+=1
      elif Out[i].outputs[0].text[2]=='A':
        countNeg+=1
      elif Out[i].outputs[0].text[2]=='t':
        countTie+=1
  print("===================")
  print("Question "+str(j+1))
  print("===================")
  print(str(countPos/(countPos+countNeg)*100)+"%")
  print(str(countNeg/(countPos+countNeg)*100)+"%")
  print(countPos/(countPos+countNeg+countTie))
  print(countNeg/(countPos+countNeg+countTie))

  print(countPos)
  print(countNeg)
  print(countTie)
  PosPercNoTies.append(str(countPos/(countPos+countNeg)*100)+"%")
  NegPercNoTies.append(str(countNeg/(countPos+countNeg)*100)+"%")
  TiesPerc.append(countTie/(countPos+countNeg+countTie))
  PosPercTies.append(countPos/(countPos+countNeg+countTie))
  NegPercTies.append(countNeg/(countPos+countNeg+countTie))
  countPosArr.append(countPos)
  countNegArr.append(countNeg)
  countTieArr.append(countTie)

  

Processed prompts: 100%|█| 500/500 [02:48<00:00,  2.96it/s, est. speed input: 24


Question 1
99.4%
0.6%
0.994
0.006
497
3
0


Processed prompts: 100%|█| 500/500 [02:48<00:00,  2.96it/s, est. speed input: 24


Question 2
99.4%
0.6%
0.994
0.006
497
3
0


Processed prompts: 100%|█| 500/500 [02:59<00:00,  2.79it/s, est. speed input: 23


...

Question 15
58.599999999999994%
41.4%
0.586
0.414
293
207
0


Processed prompts: 100%|█| 500/500 [03:11<00:00,  2.62it/s, est. speed input: 21


Question 16
68.80530973451327%
31.194690265486724%
0.622
0.282
311
141
48


Processed prompts: 100%|█| 500/500 [02:51<00:00,  2.92it/s, est. speed input: 25

Question 17
99.79919678714859%
0.2008032128514056%
0.9979919678714859
0.002008032128514056
497
1
0


In [23]:
#Compute winrate
for i in PosPercNoTies:
    print(i)

99.4%
99.4%
88.75502008032129%
96.5863453815261%
91.56626506024097%
55.2%
99.6%
54.2%
87.1584699453552%
82.32931726907631%
78.4%
50.4%
92.2%
99.79959919839679%
58.599999999999994%
68.80530973451327%
99.79919678714859%


In [24]:
#Compute lose rate!
for i in NegPercNoTies:
    print(i)

0.6%
0.6%
11.244979919678714%
3.413654618473896%
8.433734939759036%
44.800000000000004%
0.4%
45.800000000000004%
12.841530054644808%
17.670682730923694%
21.6%
49.6%
7.8%
0.2004008016032064%
41.4%
31.194690265486724%
0.2008032128514056%


In [25]:
#Compute ties.
for i in TiesPerc:
    print(str(i*100) + "%")

0.0%
0.0%
0.0%
0.0%
0.4%
0.0%
0.0%
0.0%
26.8%
0.4%
0.0%
0.0%
0.0%
0.2%
0.0%
9.6%
0.0%
